In [ ]:
import eyeviz
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
from IPython.display import display, clear_output
from matplotlib.colors import LinearSegmentedColormap
cmap1 = LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)

In [ ]:
h5f = h5py.File(eyeviz.h5files[0],'r')
def ploteyew(patient='070',eyeside='L',t=0,gridpts=640,tness='GCIPL',opacity1=1.0,opacity2=1.0):
    eyeviz.ploteye(h5f,patient,eyeside,t,gridpts,tness,opacity1,opacity2)
    
d1 = widgets.Dropdown(options=eyeviz.numsunique,description='Patient')
d2 = widgets.Dropdown(options=['L','R'],description='Eyeside')
d3 = widgets.Dropdown(options=eyeviz.thickunique,description='Layer',value='GCIPL')
d4 = widgets.Dropdown(options=eyeviz.yunique,description='Grid Pts')
s1 = widgets.IntSlider(min=0,max=len(h5f['070/L'].keys())-1)

def valued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[change.new].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[change.new+'/'+d2.value].keys())-1)
    s1.value = 0
d1.observe(valued1_changed, 'value')

def valued2_changed(change):
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif change.new not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+change.new].keys())-1)
    s1.value = 0
d2.observe(valued2_changed, 'value')

def valued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+d1.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+d2.value].keys())-1)
    s1.value = 0
d3.observe(valued3_changed, 'value')

interact(ploteyew,patient=d1,eyeside=d2,tness=d3,gridpts=d4,
         t=s1,opacity1=(0.0,1.0,0.1),opacity2=(0.0,1.0,0.1));